1. Model:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as functional

from torchvision import transforms


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)

        # Calculate the size of the output from conv2 after flattening
        self.fc_input_size = self._get_conv_output_size((3, 224, 224))

        self.fc1 = nn.Linear(self.fc_input_size, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def _get_conv_output_size(self, shape):
        # Create a dummy tensor with the shape of an input image
        dummy_input = torch.zeros(1, *shape)
        # Forward pass through conv1, pool, conv2, and pool
        dummy_output = self.pool(functional.relu(self.conv1(dummy_input)))
        dummy_output = self.pool(functional.relu(self.conv2(dummy_output)))
        # Flatten the output tensor
        return int(torch.flatten(dummy_output, 1).size(1))

    def forward(self, x):
        x = self.pool(functional.relu(self.conv1(x)))
        x = self.pool(functional.relu(self.conv2(x)))

        # flatten all dimensions except batch
        x = torch.flatten(x, 1)

        x = functional.relu(self.fc1(x))
        x = functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Define transforms for the training and testing sets
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a fixed size
    transforms.ToTensor()  # Convert images to tensor
])

Full Dataset:

* Warhol -> 182
* DaVinci -> 205
* Monet -> 498
* Renoir -> 498
* Gogh -> 650
* Rembrandt -> 655
* Degas -> 703
* Dali -> 740
* Picasso -> 992
* Munch -> 1765

Accuracy: 50%, every picture is predicted as "Munch"

Created class data_selector to limit the amount of images the model is training on. 
Images limited to 180 since only 182 Warhol images were found

Accuracy dropped to 13%

Increased number of epochs from 2 to 4, batches from 32 to 10, accuracy to 22%

Increased image size to 2000 x 2000, runtime error

Fixed bug, image size decreased to 512x512

Removed .jfif files, which are not recognised by torch

Increased Epochs to 20, loss reached low levels, accuracy still just 36%

Decreased Epochs to 10, loss decreased but not that low, accuracy dropped to 20%

![](JupiterPictures/LossPlot512x512.png)

Decreased image size to 50x50, reached 34% accuracy

Decreased image size to 32x32, increased batch size to 20, epochs to 40, reached 29% accuracy

![](JupiterPictures/LossPlot32x32.png)

Reverted batch size to 5

slighly different data set (different vanGogh i think), changed training set to 250 instead of 180. 
32% accuracy, munch random the rest either ~40% or ~20% accuracy

![](JupiterPictures/lossPlot250TrainingSize.png)



Alteration: 
    raised trianing data size to 350, warhol davinci only classes without enough data
Results:
    33% accuacy
    gogh random
    rest evenly range from 20-55%

![](JupiterPictures/LossPlot350TrainingSize.png)


Alteration; training size now 500. classes with data<500 Renoir Warhol Monet Davinci Gogh
Results:
32% accuracy
Accuracy for class: Dali  is 24.5 %
Accuracy for class: Degas is 43.0 %
Accuracy for class: Gogh  is 19.0 %
Accuracy for class: Monet is 30.2 %
Accuracy for class: Munch is 37.6 %
Accuracy for class: Picasso is 25.7 %
Accuracy for class: Rembrandt is 55.6 %
Accuracy for class: Renoir is 26.8 %
Accuracy for class: Warhol is 14.3 %
Accuracy for class: daVinci is 39.5 %

![](JupiterPictures/LossPlot500TrainingSize.png)


Alteration: training size 400, 35 epochs.

Implemented data augmentation (horizontal + vertical flip) and best epoch selection. Best epoch alone delivered 44% accuracy, with data augmentation only 41%, although all classes have similar accuracies

Added various choices for parameters to find the real best model:

batch_size_options = [5, 20, 40]

learning_rate_options = [0.01, 0.001]

Training was very slow and results with batch size 5 were similar or lower to batch size 10 (40.83% - 43.15% +), so 5 was removed

Accuracy for best model on test data was disappointing, only 15%

Reverted to single model training, reached 45% accuracy on test data with bach size 20, 10 epochs and learning rate 0.01